In [4]:
import pandas as pd
import numpy as np
import math
import math
from pathlib import Path

from tournament import Tournament, Participant, Match

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
GAME_ID = [273, 314, 331, 348, 359, 371, 380, 390, 399, 406]
# id = 273
for id in GAME_ID:

    league_settings_query = f"SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '{id}'"
    league_settings = (
        DatabaseCursor(PATH, options=OPTION_DEV)
        .copy_data_from_postgres(league_settings_query)
        .drop_duplicates()
    )

    meta_query = "SELECT game_id, league_id, end_week FROM dev.league_metadata"
    meta = DatabaseCursor(PATH).copy_data_from_postgres(meta_query).drop_duplicates()

    league_settings = league_settings.merge(
        meta,
        how="inner",
        left_on=["game_id", "league_id"],
        right_on=["game_id", "league_id"],
    )

    num_playoff_teams = league_settings["num_playoff_teams"].values[0]
    num_conso_teams = league_settings["num_playoff_consolation_teams"].values[0]
    max_teams = league_settings["max_teams"].values[0]
    num_toliet_teams = (
        (
            max_teams - num_playoff_teams
            if num_playoff_teams >= (max_teams / 2)
            else num_playoff_teams
        )
        if num_conso_teams != 0
        else False
    )
    playoff_start_week = league_settings["playoff_start_week"].values[0]
    playoff_end_week = league_settings["end_week"].values[0]
    playoff_weeks = range(playoff_start_week, playoff_end_week + 1)

    playoff_board_query = f"SELECT * FROM prod.playoff_board_{id}"
    playoff_board = (
        DatabaseCursor(PATH, options=OPTION_PROD)
        .copy_data_from_postgres(playoff_board_query)
        .drop_duplicates()
    )

    teams = list(playoff_board["team_name"].head(max_teams).unique())

    playoff_teams = list(
        playoff_board["team_name"][
            playoff_board["reg_season_rank"] <= num_playoff_teams
        ].unique()
    )
    if num_toliet_teams:
        if id == 314:
            conso_teams = list(
                playoff_board["team_name"][
                    playoff_board["reg_season_rank"] <= (max_teams - num_playoff_teams)
                ].unique()
            )
            conso_teams = [team for team in conso_teams if team not in playoff_teams]
            toilet_teams = False

        elif id >= 406:
            conso_teams = list(
                playoff_board["team_name"][
                    playoff_board["reg_season_rank"] > num_playoff_teams
                ].unique()
            )
            toilet_teams = list(
                playoff_board["team_name"][
                    playoff_board["reg_season_rank"] > (max_teams - num_conso_teams)
                ].unique()
            )
            conso_teams = [team for team in conso_teams if team not in toilet_teams]

        else:
            conso_teams = False
            toilet_teams = list(
                playoff_board["team_name"][
                    playoff_board["reg_season_rank"] > (max_teams - num_toliet_teams)
                ].unique()
            )


    else:
        toilet_teams = False
        conso_teams = False

    playoff_board.set_index(playoff_board["team_key"], drop=True, inplace=True)

    playoff_bracket = Tournament(playoff_teams)
    num_of_rounds = int(math.ceil(math.log2(len(playoff_teams))))
    for round_ in range(num_of_rounds):
        for match in playoff_bracket.get_active_matches():
            right_comp = match.get_participants()[0].get_competitor()
            right_score = playoff_board["team_pts"][
                (playoff_board["team_name"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            left_comp = match.get_participants()[1].get_competitor()
            left_score = playoff_board["team_pts"][
                (playoff_board["team_name"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
            if right_score > left_score:
                match.set_winner(right_comp)
            elif right_score < left_score:
                match.set_winner(left_comp)
    print(playoff_bracket.get_final())

    if conso_teams:
        if len(conso_teams) < len(playoff_teams):
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        print(conso_bracket.get_final())

    if toilet_teams:
        if len(toilet_teams) < len(playoff_teams):
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_name"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        print(toilet_bracket.get_final())

Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  from dev.league_settings where game_id = '273'
Successfully pulled: SELECT game_id, league_id, end_week FROM dev.league_metadata
Successfully pulled: SELECT * FROM prod.playoff_board_273
Goons: 88.5 vs Taco: 65.86
Tebowners: 106.34 vs ghost: 131.12
Bro Lo El Cunado: 71.9 vs The Fear Boners: 75.38
ELE: 77.14 vs Ruxin: 63.68
Goons: 111.66 vs ELE: 123.5
ghost: 110.48 vs The Fear Boners: 120.76
Taco: 135.44 vs Ruxin: 90.34
Tebowners: 100.7 vs Bro Lo El Cunado: 69.02
ELE: 96.02 vs The Fear Boners: 98.96
Goons: 105.22 vs ghost: 112.76
Taco: 135.6 vs Tebowners: 79.68
Ruxin: 110.12 vs Bro Lo El Cunado: 49.1
{'1': 'The Fear Boners', '2': 'ELE', '3': 'ghost', '4': 'Goons', '5': 'Taco', '6': 'Tebowners', '7': 'Ruxin', '8': 'Bro Lo El Cunado'}
Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week  

In [38]:
playoff_board

,game_id,team_key,team_name,team_nickname,week,team_pts,team_pro_pts,ttl_pts_for_run,ttl_pro_pts_for_run,reg_season_rank
team_key,,,,,,,,,,
273.l.777818.t.7,273,273.l.777818.t.7,Goons,Jake,15,88.50,106.62,88.50,106.62,1
273.l.777818.t.10,273,273.l.777818.t.10,Tebowners,273.l.777818.t.10,15,106.34,117.06,106.34,117.06,2
273.l.777818.t.2,273,273.l.777818.t.2,Bro Lo El Cunado,273.l.777818.t.2,15,71.90,58.54,71.90,58.54,3
273.l.777818.t.6,273,273.l.777818.t.6,ELE,Pete,15,77.14,90.89,77.14,90.89,4
273.l.777818.t.9,273,273.l.777818.t.9,Ruxin,273.l.777818.t.9,15,63.68,110.55,63.68,110.55,5
273.l.777818.t.4,273,273.l.777818.t.4,The Fear Boners,Wes,15,75.38,85.51,75.38,85.51,6
273.l.777818.t.12,273,273.l.777818.t.12,ghost,273.l.777818.t.12,15,131.12,96.76,131.12,96.76,7
273.l.777818.t.1,273,273.l.777818.t.1,Taco,James,15,65.86,102.73,65.86,102.73,8
273.l.777818.t.8,273,273.l.777818.t.8,Potatum,273.l.777818.t.8,15,0.00,80.35,0.00,80.35,9


In [2]:
conso_teams

NameError: name 'conso_teams' is not defined

In [40]:
toilet_teams

False

In [41]:
league_settings

,game_id,league_id,max_teams,num_playoff_consolation_teams,num_playoff_teams,playoff_start_week,end_week
0,273,777818,12,0,8,15,17


In [42]:
num_toliet_teams

False

In [43]:
num_conso_teams

0